In [1]:
import Book
import Page
import time
import RX
import numpy as np

buffer  = Book.SlidingBook(num_pages=5, page_size=5)
rx = RX.UDP_RX(buffer=buffer)

In [2]:
buffer.add_packet(np.array([0, b'message0', b'MAC', time.time()]))
buffer.pages[0].packets

array([[b'0', b'message0', b'MAC', b'1725203704.3978302'],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=object)

In [3]:
buffer.pages[0].is_full()
print(buffer.get_page_index())


page = buffer.add_packet(np.array([26, b'message1', b'MAC', time.time()]))
if page is not None:
    print(page.packets)

[0]
[[b'0' b'message0' b'MAC' b'1725203704.3978302']
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [5]:

def fill_missing_packet_in_page(page: Page.Page) -> Page.Page:
    if page.is_full():
        return page
    # Identify empty packets
    empty_rows_mask = (page.packets[:, 0] == 0)
    empty_indices = np.where(empty_rows_mask)[0]
    # Sequence numbers of existing packets
    existing_seq_nums = page.packets[~empty_rows_mask, 0].astype(int)
    # Find the missing sequence numbers 
    missing_seq_nums = np.setdiff1d(np.arange(page.page_size), existing_seq_nums, assume_unique=True)
    # If no missing sequence numbers, return the original page
    # Create an array to store the missing rows
    missing_data = np.zeros((len(missing_seq_nums), page.packets.shape[1]), dtype=object)
    # Fill the missing data with sequence numbers and timestamps
    missing_data[:, 0] = missing_seq_nums.astype(str).astype(bytes)
    missing_data[:, 3] = str(time.time()).encode()
    # Replace the empty rows with the missing data
    page.packets[empty_indices[:len(missing_seq_nums)]] = missing_data
    return page

fill_missing_packet_in_page(page).packets



array([[b'0', b'message0', b'MAC', b'1725203704.3978302'],
       [b'1', 0, 0, b'1725203704.4111786'],
       [b'2', 0, 0, b'1725203704.4111786'],
       [b'3', 0, 0, b'1725203704.4111786'],
       [b'4', 0, 0, b'1725203704.4111786']], dtype=object)

In [11]:
X = np.array([[1,0], [0,1]])

for t, p  in zip(*np.where(X == 1)):
    print(t, p)

0 0
1 1
